In [5]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
from adopt.malaria import update_ads


update_ads()

In [ ]:
from adopt.malaria import update_audience

update_audience()

In [8]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from typing import List
import json

env = Env()
env.read_env('.env-upswell-vlab', override=True)
db_conf = get_db_conf(env)

# CREATING CONFS - MNM

In [13]:
USER = "nandanmarkrao@gmail.com"
CAMPAIGN = "mnm-december"
CAMPAIGNID = "28f43eeb-4fd3-4351-b46e-a8ca3f517c55"

In [21]:
c = {'optimization_goal': 'REPLIES',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 800000.0,
     'min_budget': 10000.0,
     'opt_window': 72,
     'end_date': '2020-12-31',
     'ad_account': '2240415876083895',
     'ad_campaign': 'impact-evaluation-round-B-test'}

config = CampaignConf(**c)

create_campaign_confs([(USER, CAMPAIGN, "opt", None, json.dumps(config._asdict()))], db_conf)

In [22]:
import json

audience_confs = load_typed_json("config/audiences.json", List[AudienceConf])
confs = [(USER, CAMPAIGN, "audience", a.name, json.dumps(a._asdict())) for a in audience_confs]

create_campaign_confs(confs, db_conf)

In [23]:
from adopt.marketing import CreativeConf

creatives = load_typed_json("config/creatives.json", List[CreativeConf])
confs = [(USER, CAMPAIGN, "creative", c.name, json.dumps(c._asdict())) for c in creatives]

create_campaign_confs(confs, db_conf)

In [24]:
strata_data = [(USER, CAMPAIGN, "stratum", c["id"], json.dumps(c)) for c in strata]

create_campaign_confs(strata_data, db_conf)

# TESTING

In [26]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID)

In [31]:
from adopt.malaria import hydrate_strata, hydrate_audiences

In [ ]:
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

audiences = hydrate_audiences(df, m, audience_confs)

In [ ]:
strata = hydrate_strata(state, confs["stratum"], confs["creative"])
df = get_df(db_conf, userinfo.survey_user, strata)

In [9]:
import pandas as pd


def load_cities(path):
    cities = pd.read_csv(path)
    cities = cities[cities.rad >= 1.0]
    return cities


df = pd.read_csv('output/cities.csv')

In [10]:
df = df[['disthash', 'rad', 'lng', 'lat', 'creative_group', 'include_audience']]

In [11]:
from facebook_business.adobjects.targetinggeolocationcustomlocation \
    import TargetingGeoLocationCustomLocation
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targetinggeolocation import TargetingGeoLocation
from facebook_business.adobjects.targeting import Targeting

from adopt.marketing import make_stratum_conf

def create_location(lat, lng, rad):
    return {
        TargetingGeoLocationCustomLocation.Field.latitude: lat,
        TargetingGeoLocationCustomLocation.Field.longitude: lng,
        TargetingGeoLocationCustomLocation.Field.radius: rad,
        TargetingGeoLocationCustomLocation.Field.distance_unit: 'kilometer',
    }

def make_stratum(facebook_targeting, quota, df):
    locs = [create_location(r.lat, r.lng, r.rad) 
           for _, r in df.iterrows()]

    targeting = {
        **facebook_targeting,
        Targeting.Field.geo_locations: {
            TargetingGeoLocation.Field.location_types: ['home'],
            TargetingGeoLocation.Field.custom_locations: locs
        },
    }

    id_ = df.disthash.iloc[0]

    return { 'id': id_,
             'metadata': {'stratumid': id_},
             'facebook_targeting': targeting, 
             'creatives': ['vlab-mnm-hindi-hand-black-white', 'vlab-mnm-hindi-grains'],
             'audiences': ["vlab-mnm-kutcha-lookalike"],
             'excluded_audiences': ["vlab-mnm-respondents-panel", "vlab-mnm-respondents-cross-sectional"],
             'quota': quota, 
             'shortcodes': ['nonexisting-basleine'],
             'target_questions': [
                 {'ref': 'md:stratumid', 'op': 'equal', 'field': 'response', 'value': id_},
                 {
                     'name': 'occupation',
                     'ref': '4fc929c7-132d-49b1-a164-515e5cc9064f',
                     'op': 'answered',
                     'field': 'response',
                     'value': None
                 },
             ]}


strata = df.groupby('disthash').apply(lambda df: make_stratum({'age_min': 20}, 100, df)).tolist()


In [ ]:
audience_confs

In [ ]:


df = get_df(db_conf, userinfo.survey_user, audience_confs)

In [ ]:
df

In [14]:
from adopt.audiences import hydrate_audiences
from adopt.marketing import manage_audiences


audiences = hydrate_audiences(df, m, audience_confs)



In [ ]:
instructions = manage_audiences(state, audiences)

In [15]:
len(audiences[0].users)

5483

In [ ]:
from adopt.facebook.update import GraphUpdater
import logging

updater = GraphUpdater(state)
for i in instructions[1:2]:
    report = updater.execute(i)
    logging.info(report)

In [9]:
df = get_df(db_conf, userinfo.survey_user, strata)

In [ ]:
df

In [11]:
spend = {get_cluster_from_adset(n): i for n, i in state.spend.items()}

INFO:root:Loaded 0 adsets from campaign impact-evaluation-round-B-test


In [12]:
spend

{}

In [22]:
len(strata)

80

In [20]:
df.response.values

array(['32f58efd', 'Dhenkanal', 'langchoice', 3603030134, 1595492210172,
       '6df6b429', 'Bijnor', 'langchoice', 1486901775, 1595491918718,
       '2c1e88d9', 'Sundargarh', 'langchoice', 3842612143, 1595491121851,
       '2c1e88d9', 'Sundargarh', 'langchoice', 3704872960, 1595491162843,
       'e5f52cdf', 'Ballia', 'langchoice', 3887135341, 1595491691339,
       'b406895f', 'Bulandshahr', 'langchoice', 567776178, 1595492268040,
       'f5445658', 'Jharsuguda', 'langchoice', 4293600933, 1595492526658,
       '72d1d8e0', 'Bokaro', 'langchoice', 224924014, 1595491477305,
       '0c4a9e7c', 'Shahjahanpur', 'langchoice', 2534391942,
       1595492029164, '2e0a86c3', 'Varanasi', 'langchoice', 3496447272,
       1595492242366, '749c3a9c', 'Jaunpur', 'langchoice', 2249097819,
       1595492208751, '72d1d8e0', 'Bokaro', 'langchoice', 3575445102,
       1595491822714, '196f4fd1', 'Mathura', 'langchoice', 1597938564,
       1595491968488, '5b1466b5', 'Jhansi', 'langchoice', 3851692601,
       

In [16]:
from adopt.malaria import days_left, get_budget_lookup

budget = get_budget_lookup(
    df,
    strata,
    config.budget,
    config.min_budget,
    state.window,
    spend,
    days_left=days_left(config),
    )

In [ ]:
state.custom_audiences

In [ ]:
budget

In [29]:
from adopt.clustering import _base_budget

budget = _base_budget(strata, 2000000, 10000)

In [63]:
instructions = m.update_instructions(strata, budget)

In [ ]:
instructions

In [ ]:
from adopt.facebook.update import GraphUpdater
import logging

updater = GraphUpdater(state)
    
for i in instructions:
    report = updater.execute(i)
    logging.info(report)